In [10]:
import os
import pandas as pd

import re
import emoji
from konlpy.tag import Okt

import openai

In [12]:
# !pip install git+https://github.com/Seokhyeon-Park/hanspell.git

In [14]:
os.chdir("C:/Users/ehddl/Desktop/업무/code/sns-categorizer")

##### modify data using hanspell

In [16]:
data = pd.read_csv("tests/data/final_fine_tuning_data_original.csv", index_col=0)

In [ ]:
# import hanspell
# print(hanspell.__file__)

c:\Users\ehddl\AppData\Local\pypoetry\Cache\virtualenvs\sns-categorizer-wO1G7-CE-py3.11\Lib\site-packages\hanspell\__init__.py


In [ ]:
from hanspell import spell_checker
import xml.etree.ElementTree as ET

spelled_checked_text = []
for lst in data['media_cn_cleaned']:
    try:
        result = spell_checker.check(lst)
        spelled_text = result.checked
    except ET.ParseError:
        spelled_text = lst  # 오류 발생 시 원래 텍스트 사용
    except Exception as e:
        spelled_text = lst  # 기타 예외 발생 시 원래 텍스트 사용
        print(f"Error processing text: {lst}\nException: {e}")
    
    spelled_checked_text.append(spelled_text)

data['spelled_checked_text'] = spelled_checked_text

Error processing text: 식이섬유 1포에 6000mg 복숭아아이스티맛 식이습관 밥은 안먹어도 빵은 끊지 못하는 빵순이들을 위해 일상을 바꾸는 습관 빵을 포기못하는 당신 건강까지 챙길 수 있다면 더 완벽하지 않을까요 빵을 자주 먹다보면 포만감이 금방 사라지고 과식으로 이어지는 무한굴레 식이습관은 부족한 식이섬유를 채우며 포만감을 유지시켜 빵을 먹으면서도 배부르지만 속과 장이 편해지는 효과가 있어요 빵은 대체로 섬유질이 부족해 장건강에 영향을 줄수있는데요 식이습관은 장운동을 활성화하고 변비까지 효과적이랍니다 장건강은 체내 독소배출 및 컨디션 개선에도 중요한 역할을 해요 식이습관은 인위적인 효과를 내는 제품이 아니고 자연스러운 배변활동과 장건강을 위한 제품이예요 그렇기에 노인부터 임산부 아이들까지도 모두 섭취가능하다는거 구매는프로필링크 구매완료이벤트 오픈피드에 참여해주세요 식이습관공구 식이습관 식이섬유 바이어트 플럼잇 과일야채 유아변비 다이어터 유지어터 임산부변비 빵순이 빵스타그램
Exception: Expecting value: line 1 column 1 (char 0)
Error processing text: 라라메드 미스트 저도 같이 씁니다 하리에게 먼저 쓰기전 제 머리에 테스트해봤거든요 어떻게 쓰면 좋을지 얼마나 쓰면 좋을지 저 사실 린스도 안하고 헤어마스크도 잘 안해요귀차니즘 심함 린스나 마스크팩 하고 나면 머리 무거워지는 느낌도 싫고 그 성분들이 그리 좋은것 같지도 않고 근데 라라메드 미스트는 성분 좋은것만 가득이라 저도 써보고 싶은 욕심이 들더라구요 분사력도 좋고 향도 은은하고 촉촉하고 두비건강까지 지켜주니 같이 안쓸 이유 없지죠 요즘 제 머리카락이 건조한지 엄청 엉키는데 요 미스트쓰면 잘 풀리고 부드러워져요 머리 잘 엉키는 여아들 꼭 쓰세요 2주만 쓰시면 머리말릴때 짜증 안네요 손으로만 말려도 될 정도로 머릿결 사정없어요 하리 머리결 질투날 정도 아이들 등원길 머리 스타일링에 필수에요 머리말릴때 엉킨머리 잘 풀리도록 필수에요 지금 1

In [ ]:
# data.to_csv("final_fine_tuning_data_ver2.csv")

##### create balanced labeling data using GPT

In [18]:
data_1 = pd.read_csv("tests/data/fine_tuning_data_original.csv", index_col=0)
data_1

,acnt_id,acnt_nm,media_cn_cleaned,revised_label,label_id
0,17841400025364370,9_5jw,휴가 돌려죠,일상,19
1,17841400025364370,9_5jw,관종들 릴스 릴스타그램 릴스초보,짤/밈,21
2,17841400025364370,9_5jw,날이 좋아서,일상,19
3,17841400025364370,9_5jw,행복했던 9월 고마워,일상,19
4,17841400039750009,bichon.mayo,협찬 동결건조야채블럭 1개로 13종의 보라야채와 유산균 섭취가능 보라색 안토시아닌이...,반려동물,8
...,...,...,...,...,...
39925,17841453615191128,bong_camper83,아빠들의 일사분란함은 가족애 였다 어제 캠핑하는데 전혀 예상치 못했던 비바람 돌풍이...,홈/리빙,25
39926,17841453615191128,bong_camper83,광고 식용유 없이 전을 굽는다고 풀무원 철판수제전 3종세트 철판 바삭감자채전 철판 ...,홈/리빙,25
39927,17841453615191128,bong_camper83,제품제공 이걸 소개할 수 있어 영광입니다 가장 카고스럽게 보다 더 대담하게 카고컨테...,홈/리빙,25
39928,17841453615191128,bong_camper83,제품제공 이제 남편들이 요리 다하겠습니다 아내님들 남편에게 식스볼트 듀라박스만 사주...,홈/리빙,25


In [19]:
data_1.drop(['acnt_id', 'acnt_nm', 'media_cn_cleaned', 'label_id'], axis=1, inplace=True)

In [20]:
data_1

,revised_label
0,일상
1,짤/밈
2,일상
3,일상
4,반려동물
...,...
39925,홈/리빙
39926,홈/리빙
39927,홈/리빙
39928,홈/리빙


In [21]:
data_2 = pd.read_csv("tests/data/final_fine_tuning_data_spell_check.csv", index_col=0)
data_2

,media_cn_cleaned,label_id,spelled_checked_text
0,휴가 돌려죠,19,휴가 돌려줘
1,관종들 릴스 릴스타그램 릴스초보,21,관종들 릴스 리스타 그램 릴스 초보
2,날이 좋아서,19,날이 좋아서
3,행복했던 9월 고마워,19,행복했던 9월 고마워
4,협찬 동결건조야채블럭 1개로 13종의 보라야채와 유산균 섭취가능 보라색 안토시아닌이...,8,협찬 동결건조야채 블록 1개로 13종의 보라 야채와 유산균 섭취 가능 보라색 안토시...
...,...,...,...
39925,아빠들의 일사분란함은 가족애 였다 어제 캠핑하는데 전혀 예상치 못했던 비바람 돌풍이...,25,아빠들의 일사불란함은 가족애였다 어제 캠핑하는데 전혀 예상치 못했던 비바람 돌풍이 ...
39926,광고 식용유 없이 전을 굽는다고 풀무원 철판수제전 3종세트 철판 바삭감자채전 철판 ...,25,NaN
39927,제품제공 이걸 소개할 수 있어 영광입니다 가장 카고스럽게 보다 더 대담하게 카고컨테...,25,제품 제공 이걸 소개할 수 있어 영광입니다 가장 카고스럽게 보다 더 대담하게 카고 ...
39928,제품제공 이제 남편들이 요리 다하겠습니다 아내님들 남편에게 식스볼트 듀라박스만 사주...,25,제품 제공 이제 남편들이 요리 다하겠습니다 아내님들 남편에게 식스 볼트 듀라 박스만...


In [22]:
data_2["spelled_checked_text"] = data_2["spelled_checked_text"].fillna("")  # NaN → 빈 문자열
data_2["spelled_checked_text"] = data_2.apply(
    lambda row: row["media_cn_cleaned"] if row["spelled_checked_text"] == "" else row["spelled_checked_text"],
    axis=1)

In [23]:
data_2.drop(['media_cn_cleaned'], axis=1, inplace=True)

In [24]:
data = pd.concat([data_1, data_2], axis=1)
data

,revised_label,label_id,spelled_checked_text
0,일상,19,휴가 돌려줘
1,짤/밈,21,관종들 릴스 리스타 그램 릴스 초보
2,일상,19,날이 좋아서
3,일상,19,행복했던 9월 고마워
4,반려동물,8,협찬 동결건조야채 블록 1개로 13종의 보라 야채와 유산균 섭취 가능 보라색 안토시...
...,...,...,...
39925,홈/리빙,25,아빠들의 일사불란함은 가족애였다 어제 캠핑하는데 전혀 예상치 못했던 비바람 돌풍이 ...
39926,홈/리빙,25,광고 식용유 없이 전을 굽는다고 풀무원 철판수제전 3종세트 철판 바삭감자채전 철판 ...
39927,홈/리빙,25,제품 제공 이걸 소개할 수 있어 영광입니다 가장 카고스럽게 보다 더 대담하게 카고 ...
39928,홈/리빙,25,제품 제공 이제 남편들이 요리 다하겠습니다 아내님들 남편에게 식스 볼트 듀라 박스만...


In [25]:
data['revised_label'].value_counts()

revised_label
베이비/키즈         5883
뷰티             5742
여행/관광          5339
유명장소/핫플        5112
일상             3660
패션             2746
홈/리빙           2052
푸드             1901
다이어트/건강보조식품    1812
반려동물           1458
엔터테인먼트          744
스포츠             700
미술/디자인          420
결혼/연애           397
IT              331
교육              331
사진/영상           251
짤/밈             246
만화/애니/툰         177
자동차/모빌리티        169
셀럽              133
문구/완구           100
시사               72
게임               51
취미               29
브랜드공식계정           7
Name: count, dtype: int64

In [26]:
from dotenv import load_dotenv
load_dotenv('config/.env')

True

In [13]:
import os
openai_key = os.getenv("OPENAI_API_KEY")
os.environ['OPENAI_API_KEY'] = openai_key
os.environ['MODEL_NAME'] = 'gpt-4o'

In [18]:
# 1. 데이터 준비
underrepresented_labels = {
    "브랜드공식계정": 500,
    "취미": 500,
    "게임": 500,
    "시사": 500,
    "문구/완구": 500,
    "셀럽": 500,
    "자동차/모빌리티": 500,
    "만화/애니/툰": 500,
    "짤/밈": 500,
    "사진/영상": 500,
    "IT": 500,
    "교육": 500,
    "결혼/연애": 500,
    "미술/디자인": 500,
}

# 2. 프롬프트 생성 함수
def generate_prompt(label, example_texts, num_to_generate):
    example_str = "\n".join([f"- {text}" for text in example_texts])
    return f"""당신은 소셜 미디어 데이터를 분류하는 AI 학습 데이터를 생성하는 전문가입니다.
            아래는 "{label}" 카테고리에 속하는 실제 소셜 미디어 텍스트 예시입니다:

            {example_str}

            위와 유사한 형식과 주제를 유지하면서, "{label}" 카테고리에 적합한 새로운 소셜 미디어 텍스트 {num_to_generate}개를 생성해 주세요.
            각 문장은 실제 인스타그램에서 사람들이 사용하는 말투처럼 자연스러운 말투로 작성해주고, 해당 주제의 특징을 잘 반영해야 합니다.
            한 두-세 문장으로 각각 출력해 하되, 문장 앞에 번호나 특수문자는 붙이지 않아도 됩니다.
            
            """

# 3. GPT 호출 함수
def generate_data_with_gpt(prompt, model_name='gpt-4o', temperature=0.7):
    api_key = os.getenv("OPENAI_API_KEY")
    if not api_key:
        raise ValueError("OPENAI_API_KEY 환경변수가 설정되지 않았습니다.")
    
    openai.api_key = api_key

    try:
        response = openai.ChatCompletion.create(
            model=model_name,
            temperature=temperature,
            messages=[
                {"role": "system", "content": "너는 SNS 게시물 데이터 생성 전문가야. 카테고리별로 자연스러운 게시물을 만들어야 해."},
                {"role": "user", "content": prompt}
            ]
        )
        return response["choices"][0]["message"]["content"].strip()
    except Exception as e:
        print(f"[Error] {e}")
        return None



In [19]:
# 4. 자동 생성 루프
augmented_data = []

for label, target_count in underrepresented_labels.items():
    current_count = data[data["revised_label"] == label].shape[0]
    num_to_generate = target_count - current_count
    if num_to_generate <= 0:
        continue

    sample_texts = data[data["revised_label"] == label]["spelled_checked_text"].dropna().sample(
        min(7, current_count), random_state=42).tolist()

    prompt = generate_prompt(label, sample_texts, num_to_generate)
    print(f"[프롬프트 생성 완료] {label}: {num_to_generate}개 생성 요청")

    response = generate_data_with_gpt(prompt)
    if response:
        new_samples = [line.lstrip("- ").strip() for line in response.split("\n") if line.strip()]
        for text in new_samples:
            augmented_data.append({"text": text, "revised_label": label})

[프롬프트 생성 완료] 브랜드공식계정: 493개 생성 요청
[프롬프트 생성 완료] 취미: 471개 생성 요청
[프롬프트 생성 완료] 게임: 449개 생성 요청
[프롬프트 생성 완료] 시사: 428개 생성 요청
[프롬프트 생성 완료] 문구/완구: 400개 생성 요청
[프롬프트 생성 완료] 셀럽: 367개 생성 요청
[프롬프트 생성 완료] 자동차/모빌리티: 331개 생성 요청
[프롬프트 생성 완료] 만화/애니/툰: 323개 생성 요청
[프롬프트 생성 완료] 짤/밈: 254개 생성 요청
[프롬프트 생성 완료] 사진/영상: 249개 생성 요청
[프롬프트 생성 완료] IT: 169개 생성 요청
[프롬프트 생성 완료] 교육: 169개 생성 요청
[프롬프트 생성 완료] 결혼/연애: 103개 생성 요청
[프롬프트 생성 완료] 미술/디자인: 80개 생성 요청


In [20]:
augmented_df = pd.DataFrame(augmented_data)
print(f"총 생성된 문장 수: {len(augmented_df)}")
augmented_df

총 생성된 문장 수: 3061


,text,revised_label
0,"2024년 상반기, 미주 및 유럽 바이어들에게 전통 한식을 소개하기 위해 브랜드 홍...",브랜드공식계정
1,새로운 화장품 라인 출시를 앞두고 해외 인플루언서들에게 샘플을 보내고 있습니다. 전...,브랜드공식계정
2,올해는 한국 전통 수제 비누를 필리핀과 태국 시장에 선보일 예정입니다. 자연과 조화...,브랜드공식계정
3,"시즌 한정 컬렉션을 준비 중입니다! 이번 겨울, 프랑스와 이탈리아의 패션 트렌드를 ...",브랜드공식계정
4,차세대 전기차 부품을 북미 시장에 선보일 준비가 되어 있습니다. 지속 가능한 미래를...,브랜드공식계정
...,...,...
3056,"아름다운 색채의 향연, 수채화로 그려낸 풍경화.",미술/디자인
3057,"예술가의 아침, 커피와 함께하는 스케치 시간.",미술/디자인
3058,"디자인 스튜디오에서의 하루, 창의력 넘치는 공간.",미술/디자인
3059,"도시의 밤을 담은 사진, 빛과 그림자의 조화.",미술/디자인


In [ ]:
# augmented_df.to_csv("tests/augmented_df.csv")

In [6]:
import pandas as pd
augmented_df = pd.read_csv("C:/Users/ehddl/Desktop/업무/code/sns-categorizer/tests/data/augmented_df.csv", index_col=0)

In [7]:
augmented_df

,text,revised_label
0,"2024년 상반기, 미주 및 유럽 바이어들에게 전통 한식을 소개하기 위해 브랜드 홍...",브랜드공식계정
1,새로운 화장품 라인 출시를 앞두고 해외 인플루언서들에게 샘플을 보내고 있습니다. 전...,브랜드공식계정
2,올해는 한국 전통 수제 비누를 필리핀과 태국 시장에 선보일 예정입니다. 자연과 조화...,브랜드공식계정
3,"시즌 한정 컬렉션을 준비 중입니다! 이번 겨울, 프랑스와 이탈리아의 패션 트렌드를 ...",브랜드공식계정
4,차세대 전기차 부품을 북미 시장에 선보일 준비가 되어 있습니다. 지속 가능한 미래를...,브랜드공식계정
...,...,...
3056,"아름다운 색채의 향연, 수채화로 그려낸 풍경화.",미술/디자인
3057,"예술가의 아침, 커피와 함께하는 스케치 시간.",미술/디자인
3058,"디자인 스튜디오에서의 하루, 창의력 넘치는 공간.",미술/디자인
3059,"도시의 밤을 담은 사진, 빛과 그림자의 조화.",미술/디자인


In [8]:
label_map = {
'IT': 0,
 '게임': 1,
 '결혼/연애': 2,
 '교육': 3,
 '다이어트/건강보조식품': 4,
 '만화/애니/툰': 5,
 '문구/완구': 6,
 '미술/디자인': 7,
 '반려동물': 8,
 '베이비/키즈': 9,
 '뷰티': 10,
 '브랜드공식계정': 11,
 '사진/영상': 12,
 '셀럽': 13,
 '스포츠': 14,
 '시사': 15,
 '엔터테인먼트': 16,
 '여행/관광': 17,
 '유명장소/핫플': 18,
 '일상': 19,
 '자동차/모빌리티': 20,
 '짤/밈': 21,
 '취미': 22,
 '패션': 23,
 '푸드': 24,
 '홈/리빙': 25
}

augmented_df['label_id'] = augmented_df['revised_label'].map(label_map)

In [9]:
augmented_df.drop(['revised_label'], axis=1, inplace=True)

In [27]:
data_copy = data.copy()
data_copy.drop(['revised_label'], axis=1, inplace=True)
data_copy.rename(columns={'spelled_checked_text' : 'text'}, inplace=True)
data_copy = data_copy[['text', 'label_id']]

In [28]:
final_df = pd.concat([data_copy, augmented_df], ignore_index=True, axis=0)

In [29]:
final_df

,text,label_id
0,휴가 돌려줘,19
1,관종들 릴스 리스타 그램 릴스 초보,21
2,날이 좋아서,19
3,행복했던 9월 고마워,19
4,협찬 동결건조야채 블록 1개로 13종의 보라 야채와 유산균 섭취 가능 보라색 안토시...,8
...,...,...
42919,"아름다운 색채의 향연, 수채화로 그려낸 풍경화.",7
42920,"예술가의 아침, 커피와 함께하는 스케치 시간.",7
42921,"디자인 스튜디오에서의 하루, 창의력 넘치는 공간.",7
42922,"도시의 밤을 담은 사진, 빛과 그림자의 조화.",7


In [30]:
final_df.to_csv("tests/data/final_fine_tuning_augmented_data.csv")

create multi-labeled data

In [31]:
augmented_data = pd.read_csv("tests/data/augmented_df.csv", index_col=0)
augmented_data

,text,revised_label
0,"2024년 상반기, 미주 및 유럽 바이어들에게 전통 한식을 소개하기 위해 브랜드 홍...",브랜드공식계정
1,새로운 화장품 라인 출시를 앞두고 해외 인플루언서들에게 샘플을 보내고 있습니다. 전...,브랜드공식계정
2,올해는 한국 전통 수제 비누를 필리핀과 태국 시장에 선보일 예정입니다. 자연과 조화...,브랜드공식계정
3,"시즌 한정 컬렉션을 준비 중입니다! 이번 겨울, 프랑스와 이탈리아의 패션 트렌드를 ...",브랜드공식계정
4,차세대 전기차 부품을 북미 시장에 선보일 준비가 되어 있습니다. 지속 가능한 미래를...,브랜드공식계정
...,...,...
3056,"아름다운 색채의 향연, 수채화로 그려낸 풍경화.",미술/디자인
3057,"예술가의 아침, 커피와 함께하는 스케치 시간.",미술/디자인
3058,"디자인 스튜디오에서의 하루, 창의력 넘치는 공간.",미술/디자인
3059,"도시의 밤을 담은 사진, 빛과 그림자의 조화.",미술/디자인


In [32]:
data_1 = pd.read_csv("tests/data/3rd_labeling_data.csv", index_col=0)
data_1

,acnt_id,acnt_nm,acnt_sub_nm_cleaned,intro_txt_cleaned,media_cn_cleaned,label_list,label,clean_label,revised_label
0,17841400025364370,9_5jw,3 10 1,NaN,휴가 돌려죠,['일상'],일상,일상,일상
1,17841400025364370,9_5jw,3 10 1,NaN,관종들 릴스 릴스타그램 릴스초보,"['짤/밈', '엔터테인먼트', '일상']",짤/밈,짤/밈,짤/밈
2,17841400025364370,9_5jw,3 10 1,NaN,날이 좋아서,['일상'],일상,일상,일상
3,17841400025364370,9_5jw,3 10 1,NaN,행복했던 9월 고마워,['일상'],일상,일상,일상
4,17841400039750009,bichon.mayo,뚱마요 mayo,11짤 주세요 만렙 개인기 다양함 네이버 펫인플루언서 뚱마요 엄마는 kkf 애견미용...,협찬 동결건조야채블럭 1개로 13종의 보라야채와 유산균 섭취가능 보라색 안토시아닌이...,"['반려동물', '푸드', '일상']",반려동물,반려동물,반려동물
...,...,...,...,...,...,...,...,...,...
39925,17841453615191128,bong_camper83,봉캠퍼 잘봐요 매번 신선한지캠핑캠핑용품 리뷰,좌충우돌 2인조 캠핑크루 jb jamesjuncamp bongcamper83 amb...,아빠들의 일사분란함은 가족애 였다 어제 캠핑하는데 전혀 예상치 못했던 비바람 돌풍이...,"['홈/리빙', '일상', '여행/관광']",홈/리빙,홈/리빙,홈/리빙
39926,17841453615191128,bong_camper83,봉캠퍼 잘봐요 매번 신선한지캠핑캠핑용품 리뷰,좌충우돌 2인조 캠핑크루 jb jamesjuncamp bongcamper83 amb...,광고 식용유 없이 전을 굽는다고 풀무원 철판수제전 3종세트 철판 바삭감자채전 철판 ...,"['홈/리빙', '일상', '푸드']",홈/리빙,홈/리빙,홈/리빙
39927,17841453615191128,bong_camper83,봉캠퍼 잘봐요 매번 신선한지캠핑캠핑용품 리뷰,좌충우돌 2인조 캠핑크루 jb jamesjuncamp bongcamper83 amb...,제품제공 이걸 소개할 수 있어 영광입니다 가장 카고스럽게 보다 더 대담하게 카고컨테...,"['홈/리빙', '일상', '여행/관광']",홈/리빙,홈/리빙,홈/리빙
39928,17841453615191128,bong_camper83,봉캠퍼 잘봐요 매번 신선한지캠핑캠핑용품 리뷰,좌충우돌 2인조 캠핑크루 jb jamesjuncamp bongcamper83 amb...,제품제공 이제 남편들이 요리 다하겠습니다 아내님들 남편에게 식스볼트 듀라박스만 사주...,"['홈/리빙', '일상', '취미']",홈/리빙,홈/리빙,홈/리빙


In [33]:
data_1.drop(['acnt_id', 'acnt_nm', 'acnt_sub_nm_cleaned', 'intro_txt_cleaned', 'label', 'clean_label', 'revised_label'], axis=1, inplace=True)

In [34]:
data_1

,media_cn_cleaned,label_list
0,휴가 돌려죠,['일상']
1,관종들 릴스 릴스타그램 릴스초보,"['짤/밈', '엔터테인먼트', '일상']"
2,날이 좋아서,['일상']
3,행복했던 9월 고마워,['일상']
4,협찬 동결건조야채블럭 1개로 13종의 보라야채와 유산균 섭취가능 보라색 안토시아닌이...,"['반려동물', '푸드', '일상']"
...,...,...
39925,아빠들의 일사분란함은 가족애 였다 어제 캠핑하는데 전혀 예상치 못했던 비바람 돌풍이...,"['홈/리빙', '일상', '여행/관광']"
39926,광고 식용유 없이 전을 굽는다고 풀무원 철판수제전 3종세트 철판 바삭감자채전 철판 ...,"['홈/리빙', '일상', '푸드']"
39927,제품제공 이걸 소개할 수 있어 영광입니다 가장 카고스럽게 보다 더 대담하게 카고컨테...,"['홈/리빙', '일상', '여행/관광']"
39928,제품제공 이제 남편들이 요리 다하겠습니다 아내님들 남편에게 식스볼트 듀라박스만 사주...,"['홈/리빙', '일상', '취미']"


In [35]:
data_2 = pd.read_csv("tests/data/final_fine_tuning_data_spell_check.csv", index_col=0)

In [36]:
data_2["spelled_checked_text"] = data_2["spelled_checked_text"].fillna("")  # NaN → 빈 문자열
data_2["spelled_checked_text"] = data_2.apply(
    lambda row: row["media_cn_cleaned"] if row["spelled_checked_text"] == "" else row["spelled_checked_text"],
    axis=1)

In [37]:
data_2

,media_cn_cleaned,label_id,spelled_checked_text
0,휴가 돌려죠,19,휴가 돌려줘
1,관종들 릴스 릴스타그램 릴스초보,21,관종들 릴스 리스타 그램 릴스 초보
2,날이 좋아서,19,날이 좋아서
3,행복했던 9월 고마워,19,행복했던 9월 고마워
4,협찬 동결건조야채블럭 1개로 13종의 보라야채와 유산균 섭취가능 보라색 안토시아닌이...,8,협찬 동결건조야채 블록 1개로 13종의 보라 야채와 유산균 섭취 가능 보라색 안토시...
...,...,...,...
39925,아빠들의 일사분란함은 가족애 였다 어제 캠핑하는데 전혀 예상치 못했던 비바람 돌풍이...,25,아빠들의 일사불란함은 가족애였다 어제 캠핑하는데 전혀 예상치 못했던 비바람 돌풍이 ...
39926,광고 식용유 없이 전을 굽는다고 풀무원 철판수제전 3종세트 철판 바삭감자채전 철판 ...,25,광고 식용유 없이 전을 굽는다고 풀무원 철판수제전 3종세트 철판 바삭감자채전 철판 ...
39927,제품제공 이걸 소개할 수 있어 영광입니다 가장 카고스럽게 보다 더 대담하게 카고컨테...,25,제품 제공 이걸 소개할 수 있어 영광입니다 가장 카고스럽게 보다 더 대담하게 카고 ...
39928,제품제공 이제 남편들이 요리 다하겠습니다 아내님들 남편에게 식스볼트 듀라박스만 사주...,25,제품 제공 이제 남편들이 요리 다하겠습니다 아내님들 남편에게 식스 볼트 듀라 박스만...


In [38]:
df1_unique = data_1.drop_duplicates(subset='media_cn_cleaned')
df2_unique = data_2.drop_duplicates(subset='media_cn_cleaned')

merged_df = pd.merge(df1_unique, df2_unique, on='media_cn_cleaned', how='inner')

In [39]:
# merged_df.drop(['media_cn_cleaned'], axis=1, inplace=True)
merged_df.rename(columns={'spelled_checked_text' : 'text', 'label_list':'multi_label'}, inplace=True)

In [40]:
label_map = {
    'IT': 0,
 '게임': 1,
 '결혼/연애': 2,
 '교육': 3,
 '다이어트/건강보조식품': 4,
 '만화/애니/툰': 5,
 '문구/완구': 6,
 '미술/디자인': 7,
 '반려동물': 8,
 '베이비/키즈': 9,
 '뷰티': 10,
 '브랜드공식계정': 11,
 '사진/영상': 12,
 '셀럽': 13,
 '스포츠': 14,
 '시사': 15,
 '엔터테인먼트': 16,
 '여행/관광': 17,
 '유명장소/핫플': 18,
 '일상': 19,
 '자동차/모빌리티': 20,
 '짤/밈': 21,
 '취미': 22,
 '패션': 23,
 '푸드': 24,
 '홈/리빙': 25
}

inv_label_map = {v: k for k, v in label_map.items()}
merged_df['single_label'] = merged_df['label_id'].map(inv_label_map)

In [41]:
merged_df = merged_df[['text', 'single_label', 'label_id', 'multi_label']]
merged_df

,text,single_label,label_id,multi_label
0,휴가 돌려줘,일상,19,['일상']
1,관종들 릴스 리스타 그램 릴스 초보,짤/밈,21,"['짤/밈', '엔터테인먼트', '일상']"
2,날이 좋아서,일상,19,['일상']
3,행복했던 9월 고마워,일상,19,['일상']
4,협찬 동결건조야채 블록 1개로 13종의 보라 야채와 유산균 섭취 가능 보라색 안토시...,반려동물,8,"['반려동물', '푸드', '일상']"
...,...,...,...,...
39818,아빠들의 일사불란함은 가족애였다 어제 캠핑하는데 전혀 예상치 못했던 비바람 돌풍이 ...,홈/리빙,25,"['홈/리빙', '일상', '여행/관광']"
39819,광고 식용유 없이 전을 굽는다고 풀무원 철판수제전 3종세트 철판 바삭감자채전 철판 ...,홈/리빙,25,"['홈/리빙', '일상', '푸드']"
39820,제품 제공 이걸 소개할 수 있어 영광입니다 가장 카고스럽게 보다 더 대담하게 카고 ...,홈/리빙,25,"['홈/리빙', '일상', '여행/관광']"
39821,제품 제공 이제 남편들이 요리 다하겠습니다 아내님들 남편에게 식스 볼트 듀라 박스만...,홈/리빙,25,"['홈/리빙', '일상', '취미']"


In [ ]:
# merged_df.to_csv("tests/data/final_fine-tuning_multi-label_data.csv")

create multi-column data

In [44]:
augmented_data = pd.read_csv("tests/data/augmented_df.csv", index_col=0)
augmented_data

,text,revised_label
0,"2024년 상반기, 미주 및 유럽 바이어들에게 전통 한식을 소개하기 위해 브랜드 홍...",브랜드공식계정
1,새로운 화장품 라인 출시를 앞두고 해외 인플루언서들에게 샘플을 보내고 있습니다. 전...,브랜드공식계정
2,올해는 한국 전통 수제 비누를 필리핀과 태국 시장에 선보일 예정입니다. 자연과 조화...,브랜드공식계정
3,"시즌 한정 컬렉션을 준비 중입니다! 이번 겨울, 프랑스와 이탈리아의 패션 트렌드를 ...",브랜드공식계정
4,차세대 전기차 부품을 북미 시장에 선보일 준비가 되어 있습니다. 지속 가능한 미래를...,브랜드공식계정
...,...,...
3056,"아름다운 색채의 향연, 수채화로 그려낸 풍경화.",미술/디자인
3057,"예술가의 아침, 커피와 함께하는 스케치 시간.",미술/디자인
3058,"디자인 스튜디오에서의 하루, 창의력 넘치는 공간.",미술/디자인
3059,"도시의 밤을 담은 사진, 빛과 그림자의 조화.",미술/디자인


In [45]:
data_1 = pd.read_csv("tests/data/3rd_labeling_data.csv", index_col=0)
data_1

,acnt_id,acnt_nm,acnt_sub_nm_cleaned,intro_txt_cleaned,media_cn_cleaned,label_list,label,clean_label,revised_label
0,17841400025364370,9_5jw,3 10 1,NaN,휴가 돌려죠,['일상'],일상,일상,일상
1,17841400025364370,9_5jw,3 10 1,NaN,관종들 릴스 릴스타그램 릴스초보,"['짤/밈', '엔터테인먼트', '일상']",짤/밈,짤/밈,짤/밈
2,17841400025364370,9_5jw,3 10 1,NaN,날이 좋아서,['일상'],일상,일상,일상
3,17841400025364370,9_5jw,3 10 1,NaN,행복했던 9월 고마워,['일상'],일상,일상,일상
4,17841400039750009,bichon.mayo,뚱마요 mayo,11짤 주세요 만렙 개인기 다양함 네이버 펫인플루언서 뚱마요 엄마는 kkf 애견미용...,협찬 동결건조야채블럭 1개로 13종의 보라야채와 유산균 섭취가능 보라색 안토시아닌이...,"['반려동물', '푸드', '일상']",반려동물,반려동물,반려동물
...,...,...,...,...,...,...,...,...,...
39925,17841453615191128,bong_camper83,봉캠퍼 잘봐요 매번 신선한지캠핑캠핑용품 리뷰,좌충우돌 2인조 캠핑크루 jb jamesjuncamp bongcamper83 amb...,아빠들의 일사분란함은 가족애 였다 어제 캠핑하는데 전혀 예상치 못했던 비바람 돌풍이...,"['홈/리빙', '일상', '여행/관광']",홈/리빙,홈/리빙,홈/리빙
39926,17841453615191128,bong_camper83,봉캠퍼 잘봐요 매번 신선한지캠핑캠핑용품 리뷰,좌충우돌 2인조 캠핑크루 jb jamesjuncamp bongcamper83 amb...,광고 식용유 없이 전을 굽는다고 풀무원 철판수제전 3종세트 철판 바삭감자채전 철판 ...,"['홈/리빙', '일상', '푸드']",홈/리빙,홈/리빙,홈/리빙
39927,17841453615191128,bong_camper83,봉캠퍼 잘봐요 매번 신선한지캠핑캠핑용품 리뷰,좌충우돌 2인조 캠핑크루 jb jamesjuncamp bongcamper83 amb...,제품제공 이걸 소개할 수 있어 영광입니다 가장 카고스럽게 보다 더 대담하게 카고컨테...,"['홈/리빙', '일상', '여행/관광']",홈/리빙,홈/리빙,홈/리빙
39928,17841453615191128,bong_camper83,봉캠퍼 잘봐요 매번 신선한지캠핑캠핑용품 리뷰,좌충우돌 2인조 캠핑크루 jb jamesjuncamp bongcamper83 amb...,제품제공 이제 남편들이 요리 다하겠습니다 아내님들 남편에게 식스볼트 듀라박스만 사주...,"['홈/리빙', '일상', '취미']",홈/리빙,홈/리빙,홈/리빙


In [46]:
data_1.drop(['acnt_id', 'acnt_nm', 'label', 'clean_label'], axis=1, inplace=True)

In [48]:
data_2 = pd.read_csv("tests/data/final_fine_tuning_data_spell_check.csv", index_col=0)

In [49]:
data_2["spelled_checked_text"] = data_2["spelled_checked_text"].fillna("")  # NaN → 빈 문자열
data_2["spelled_checked_text"] = data_2.apply(
    lambda row: row["media_cn_cleaned"] if row["spelled_checked_text"] == "" else row["spelled_checked_text"],
    axis=1)

In [50]:
data_2

,media_cn_cleaned,label_id,spelled_checked_text
0,휴가 돌려죠,19,휴가 돌려줘
1,관종들 릴스 릴스타그램 릴스초보,21,관종들 릴스 리스타 그램 릴스 초보
2,날이 좋아서,19,날이 좋아서
3,행복했던 9월 고마워,19,행복했던 9월 고마워
4,협찬 동결건조야채블럭 1개로 13종의 보라야채와 유산균 섭취가능 보라색 안토시아닌이...,8,협찬 동결건조야채 블록 1개로 13종의 보라 야채와 유산균 섭취 가능 보라색 안토시...
...,...,...,...
39925,아빠들의 일사분란함은 가족애 였다 어제 캠핑하는데 전혀 예상치 못했던 비바람 돌풍이...,25,아빠들의 일사불란함은 가족애였다 어제 캠핑하는데 전혀 예상치 못했던 비바람 돌풍이 ...
39926,광고 식용유 없이 전을 굽는다고 풀무원 철판수제전 3종세트 철판 바삭감자채전 철판 ...,25,광고 식용유 없이 전을 굽는다고 풀무원 철판수제전 3종세트 철판 바삭감자채전 철판 ...
39927,제품제공 이걸 소개할 수 있어 영광입니다 가장 카고스럽게 보다 더 대담하게 카고컨테...,25,제품 제공 이걸 소개할 수 있어 영광입니다 가장 카고스럽게 보다 더 대담하게 카고 ...
39928,제품제공 이제 남편들이 요리 다하겠습니다 아내님들 남편에게 식스볼트 듀라박스만 사주...,25,제품 제공 이제 남편들이 요리 다하겠습니다 아내님들 남편에게 식스 볼트 듀라 박스만...


In [51]:
df1_unique = data_1.drop_duplicates(subset='media_cn_cleaned')
df2_unique = data_2.drop_duplicates(subset='media_cn_cleaned')

merged_df = pd.merge(df1_unique, df2_unique, on='media_cn_cleaned', how='inner')

In [ ]:
merged_df.drop(['media_cn_cleaned'], axis=1, inplace=True)
merged_df.rename(columns={'spelled_checked_text' : 'text', 'revised_label':'single_label'}, inplace=True)

In [58]:
merged_df

,acnt_sub_nm_cleaned,intro_txt_cleaned,label_list,single_label,label_id,text
0,3 10 1,NaN,['일상'],일상,19,휴가 돌려줘
1,3 10 1,NaN,"['짤/밈', '엔터테인먼트', '일상']",짤/밈,21,관종들 릴스 리스타 그램 릴스 초보
2,3 10 1,NaN,['일상'],일상,19,날이 좋아서
3,3 10 1,NaN,['일상'],일상,19,행복했던 9월 고마워
4,뚱마요 mayo,11짤 주세요 만렙 개인기 다양함 네이버 펫인플루언서 뚱마요 엄마는 kkf 애견미용...,"['반려동물', '푸드', '일상']",반려동물,8,협찬 동결건조야채 블록 1개로 13종의 보라 야채와 유산균 섭취 가능 보라색 안토시...
...,...,...,...,...,...,...
39818,봉캠퍼 잘봐요 매번 신선한지캠핑캠핑용품 리뷰,좌충우돌 2인조 캠핑크루 jb jamesjuncamp bongcamper83 amb...,"['홈/리빙', '일상', '여행/관광']",홈/리빙,25,아빠들의 일사불란함은 가족애였다 어제 캠핑하는데 전혀 예상치 못했던 비바람 돌풍이 ...
39819,봉캠퍼 잘봐요 매번 신선한지캠핑캠핑용품 리뷰,좌충우돌 2인조 캠핑크루 jb jamesjuncamp bongcamper83 amb...,"['홈/리빙', '일상', '푸드']",홈/리빙,25,광고 식용유 없이 전을 굽는다고 풀무원 철판수제전 3종세트 철판 바삭감자채전 철판 ...
39820,봉캠퍼 잘봐요 매번 신선한지캠핑캠핑용품 리뷰,좌충우돌 2인조 캠핑크루 jb jamesjuncamp bongcamper83 amb...,"['홈/리빙', '일상', '여행/관광']",홈/리빙,25,제품 제공 이걸 소개할 수 있어 영광입니다 가장 카고스럽게 보다 더 대담하게 카고 ...
39821,봉캠퍼 잘봐요 매번 신선한지캠핑캠핑용품 리뷰,좌충우돌 2인조 캠핑크루 jb jamesjuncamp bongcamper83 amb...,"['홈/리빙', '일상', '취미']",홈/리빙,25,제품 제공 이제 남편들이 요리 다하겠습니다 아내님들 남편에게 식스 볼트 듀라 박스만...


In [59]:
merged_df = merged_df[['acnt_sub_nm_cleaned', 'intro_txt_cleaned', 'text', 'single_label', 'label_id', 'label_list']]

In [60]:
merged_df.to_csv("tests/data/final_fine-tuning_multi-columns_data.csv")